Se usan los datos de la carpeta Data_football_mix.

En main están las temporadas desde 93_94 hasta 23_24.

En la carpeta de cada temporada están los archivos divididos por liga.

D1: Bundesliga

In [3]:
import pandas as pd
import numpy as np

In [41]:
#Carpeta Main
folder = 'Data_football_mix/main'
#Carpeta de las temporadas
folder_seasons = []
for i in range(1994, 2024):
    folder_seasons.append(f'{str(i)[2:]}_{str(i+1)[2:]}')

In [63]:
#budesliga
bundesliga_seasons = []
for season in folder_seasons:
    try:
        bundesliga_seasons.append(pd.read_csv(f'{folder}/{season}/D1.csv', sep=',', encoding='Latin1'))
    except:
        print(f'Error en {folder}/{season}/D1.csv')


Error en Data_football_mix/main/03_04/D1.csv


In [64]:
#cada temporada es un dataframe
dfs_seasons = []
for season in bundesliga_seasons:
    dfs_seasons.append(pd.DataFrame(season))

#Nos importan las columnas Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR
dfs_seasons = [season[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']] for season in dfs_seasons]

dfs_seasons[-1].head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,18/08/2023,Werder Bremen,Bayern Munich,0,4,A
1,19/08/2023,Augsburg,M'gladbach,4,4,D
2,19/08/2023,Hoffenheim,Freiburg,1,2,A
3,19/08/2023,Leverkusen,RB Leipzig,3,2,H
4,19/08/2023,Stuttgart,Bochum,5,0,H


### manejo de datos

Agregamos nuevas columnas

In [65]:
# agregar Wk a cada fila de cada temporada
# Cada equipo juega 34 partidos por temporada

for i in range(len(dfs_seasons)):
    #cada vez que los 18 equipos juegan 1 partido, se aumenta el contador
    wk = 1
    counter = 1
    for j in range(len(dfs_seasons[i])):
        if counter == 10:
            counter = 1
            wk += 1
        dfs_seasons[i].loc[j, 'Wk'] = wk
        counter += 1

dfs_seasons[-1].head()


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Wk
0,18/08/2023,Werder Bremen,Bayern Munich,0,4,A,1.0
1,19/08/2023,Augsburg,M'gladbach,4,4,D,1.0
2,19/08/2023,Hoffenheim,Freiburg,1,2,A,1.0
3,19/08/2023,Leverkusen,RB Leipzig,3,2,H,1.0
4,19/08/2023,Stuttgart,Bochum,5,0,H,1.0


In [66]:
dfs_seasons[16].tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Wk
301,05/05/12,Hertha,Hoffenheim,3,1,H,34.0
302,05/05/12,Mainz,M'gladbach,0,3,A,34.0
303,05/05/12,Nurnberg,Leverkusen,1,4,A,34.0
304,05/05/12,Stuttgart,Wolfsburg,3,2,H,34.0
305,05/05/12,Werder Bremen,Schalke 04,2,3,A,34.0


In [67]:
#rename columns
#HomeTeam -> Team_home, AwayTeam -> Team_away, FTHG -> Goals_home, FTAG -> Goals_away, 
for season in dfs_seasons:
    season.columns = ['Date', 'Team_home', 'Team_away', 'Goals_home', 'Goals_away', 'FTR', 'Wk']
    season.drop(columns={'FTR', 'Date'}, inplace=True)

dfs_seasons[-1].head()

,Team_home,Team_away,Goals_home,Goals_away,Wk
0,Werder Bremen,Bayern Munich,0,4,1.0
1,Augsburg,M'gladbach,4,4,1.0
2,Hoffenheim,Freiburg,1,2,1.0
3,Leverkusen,RB Leipzig,3,2,1.0
4,Stuttgart,Bochum,5,0,1.0


In [ ]:
def remove_suffix(item):
    return item.replace("_home", "").replace("_away", "")

column = [remove_suffix(item) for item in dfs_seasons[0].columns]
columnas = []
[columnas.append(x) for x in column if x not in columnas]
columnas.remove("Wk")
columnas.remove("Date")
columnas.remove("Team")
columnas[0] = "score_prom"
print(columnas)
print(len(columnas))